In [ ]:
# Implementing the "Cube" approach

In [1]:
import pykep as pk
import numpy as np
import json
import pickle as pkl

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib notebook

import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

# added for the Cube approach implementation
from collections import defaultdict
import time
import sgp4
from sgp4.api import Satrec, SatrecArray

In [2]:
with open("data/debris_simulation_ic.pk", "rb") as file:
    r_ic,v_ic,c_radius,to_satcat,satcat,debris = pkl.load(file)

In [3]:
def cubes(cartesian_points, cube_dimension = 10):
    retval = []
    pos = cartesian_points # km
    pos = pos / cube_dimension
    cubes = defaultdict(list)
    pos = np.floor(pos).astype(int)
    # We fill the bins
    for i, xyz in enumerate(pos):
        cubes[tuple(xyz)].append(i)
    # We find te bins with more than one entry
    for key in cubes:
        if len(cubes[key]) > 1:
            retval.append(cubes[key])
    return retval

In [5]:
def simulate_sgp4(debris, sim_time=20,time_grid=5, t0 = 8073.603992389981):
    # This list will contain all the sgp4 Satrec objects
    satellite_l = []
    for deb in debris:
        l1 = deb.line1
        l2 = deb.line2
        satellite_l.append(Satrec.twoline2rv(l1, l2))
    # Here we build the vectorized version allowing for speed
    satellites = SatrecArray(satellite_l)
    jd0, fr = pk.epoch(t0).jd, 0.0
    # The Julian dates are from jd0 to 20 years after
    jds = jd0 + np.arange(0,sim_time*365.25/time_grid)*time_grid
    frs = jds * 0
    return satellites.sgp4(jds, frs)

In [6]:
%timeit cubes(r_ic / 1000, 10)

85.6 ms ± 9.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
start = time.time()
e,r,v = simulate_sgp4(np.array(debris)[to_satcat], sim_time=20,time_grid=5, t0 = 8073.603992389981)
end = time.time()
print("Time elasped: ", end - start)

Time elasped:  26.725625038146973


In [69]:
# We assume all satellites are valid at the starting epoch
undecayed = set(np.arange(r.shape[0]))
for i in range(r.shape[1]):
    # If signalled from the sgp4, we remove the indices of the decayed satellites
    decayed = set(np.where(e[:,i]>0)[0])
    undecayed = undecayed - decayed
    # We detect all satellites couples in the same cube of 10 km size
    collision = cubes(r[np.array([i for i in undecayed]),i,:], cube_dimension = 10)
    for pair in collision:
        idx1 = pair[0]
        idx2 = pair[1]
        

[[13372, 14800], [14041, 14045], [14947, 14950], [16384, 16849], [16782, 16801], [17917, 17918], [18146, 18174], [18459, 18467], [18460, 18464], [18480, 18483], [18485, 18487], [18629, 18701]]
[[7508, 7582], [14662, 15013], [15616, 15617], [15720, 15800], [17600, 17606], [17602, 17604], [18094, 18808], [18443, 18455], [18449, 18461], [18473, 18482], [18628, 18632], [18644, 18647], [18649, 18652], [18723, 18746]]
[[17905, 17911], [18449, 18485], [18472, 18484], [18477, 18486], [18649, 18652]]
[[7538, 9580], [14978, 16150], [15070, 15964], [18532, 19013]]
[[11270, 19164], [17406, 17407], [17602, 17607], [18670, 18672]]
[[7538, 9580], [17293, 17294], [18623, 18624], [18630, 18632]]
[[11303, 11442], [17115, 17122], [18223, 18800], [18419, 18440]]
[[6138, 6371], [12156, 15167], [15223, 17068], [16394, 16395], [16967, 16971]]
[[6949, 12507], [15119, 18019], [16640, 16641], [17232, 17256], [17641, 18066], [17746, 17907], [18577, 18579]]
[[138, 9561], [11861, 12481], [13922, 16133], [15811, 15

KeyboardInterrupt: 

In [58]:
undecayed = set(np.arange(r.shape[0]))


In [68]:
[i for i in undecayed]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
